In [433]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os

In [434]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
print("fileam: ", fileam)
filepm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 5 P.M..XLSX"
print("filepm: ", filepm)
file3pm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 3 P.M..XLSX"
print("file3pm: ", file3pm)
fileyest = "OT " + (datetime.today() - timedelta(days = 1)).strftime('%d.%m.%Y' + " 5 P.M..XLSX")
print("fileyest: ", fileyest)
main_path = "C:\\Users\\mruiz\\Documents\\DB OT\\" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
data_3pm_excel = {}
data_pm_excel = {}

fileam:  OT 03.11.2022.XLSX
filepm:  OT 03.11.2022 5 P.M..XLSX
file3pm:  OT 03.11.2022 3 P.M..XLSX
fileyest:  OT 02.11.2022 5 P.M..XLSX


In [435]:
def get_data(entry_data):
    file = main_path + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1")
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [436]:
while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        break

Los datos del archivo  OT 02.11.2022 5 P.M..XLSX  fueron procesados
Los datos del archivo  OT 03.11.2022.XLSX  fueron procesados
Hay al menos un archivo que no fue leído: 
Archivo Ayer :  <class 'pandas.core.frame.DataFrame'>
Archivo hoy por la mañana:  <class 'pandas.core.frame.DataFrame'>
Archivo hoy 3 pm :  <class 'dict'>
Archivo hoy 5 pm :  <class 'dict'>


In [ ]:
"""while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
        data_3pm_excel = get_data(file3pm)
        data_pm_excel = get_data(filepm)
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        print("Archivo hoy 3 pm : ", type(data_3pm_excel))
        print("Archivo hoy 5 pm : ",type(data_pm_excel))
        break"""

In [437]:
if type(data_pm_excel) == type(data_yesterday_excel):
    programa = data_pm_excel
    print("Los datos de la tarde fueron seleccionados")
elif type(data_3pm_excel) == type(data_yesterday_excel):
    programa = data_3pm_excel
    print("Los datos del mediodia fueron seleccionados")
elif type(data_am_excel) == type(data_yesterday_excel):
    programa = data_am_excel
    print("Los datos de la mañana fueron seleccionados")
else :
    print("Error")

Los datos de la mañana fueron seleccionados


In [438]:
data2 =current_path + "\\Master_OT.xlsx"
nombres = pd.read_excel(data2, sheet_name="Names", header=0)

In [439]:
data3 = current_path + "\\Politicas.XLSX"
politicas = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [440]:
data4 = current_path + "\\Materiales_Locales.xlsx"
mlocal = pd.read_excel(data4, sheet_name="ENTREGADO", header=0)

In [441]:
data5 = current_path + "\\Materiales_Foraneos.xlsx"
mforaneo = pd.read_excel(data5, sheet_name="MATRIZ FINAL", header=1)

In [442]:
data6 = current_path + "\\Pendiente_facturar.xlsx"
reportes = pd.read_excel(data6, sheet_name="CONTROL", header=0)

c:\Users\mruiz\Documents\pythonscripts\env\lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell B3512 is marked as a date but the serial value 6692247 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [443]:
data7 = current_path + "\\Ubicaciones_Equipos.xlsx"
ubicaciones = pd.read_excel(data7, sheet_name="Sheet1", header=0)

In [492]:
data8 = current_path + "\\Reprogramaciones.xlsx"
reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)

In [444]:
reprogramaciones = data_yesterday_excel.copy()

In [445]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_reprogramada'}, inplace=True)

In [446]:
programa.rename(columns={'Inicio programado' : 'Inicio_programado', 'Plan mant.preventivo' : 'Plan_Mtto',
       'Pto.tbjo.responsable': 'Responsable', 'Status del sistema' : 'Status', 'Texto breve' : 'Descripción',
       'Denominación de objeto técnico' : 'Cliente_Equipo', 'Número de serie': 'Número_de_serie', 'Clase de orden':'Clase_de_orden',
       'Fe.liberación real':'Fecha_Liberación', 'Fecha inicio real':'Fecha_creación'}, inplace = True)

In [447]:
programa['Material'] = np.where((programa['Clase_de_orden'] == 'ZMCP') &  (programa['Descripción'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(programa['Clase_de_orden'] == 'ZPM2','SI',
np.where(programa['Clase_de_orden'] == 'ZMCP','SI',
np.where(programa['Clase_de_orden'] == 'ZMCI', 'SI',
np.where((programa['Clase_de_orden'] == 'ZPM1') &  (programa['Descripción'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [448]:
#print('Las columnas de frame 1 son', programa.columns)
#print('Las columnas de frame 2 son', nombres.columns)
#print('Las columnas de frame 3 son', politicas.columns)
#print('Las columnas de frame 4 son', mlocal.columns)
#print('Las columnas de frame 5 son', mforaneo.columns)
#print('Las columnas de reprogramaciones son ',reprogramaciones)
#print('Las columnas de frame 6 son', reportes.columns)
#print('Las columnas de frame 7 son', ubicaciones.columns)

Las columnas de frame 7 son Index(['Equipo', 'Denominación de objeto técnico', 'Cliente actual',
       'Denominación de la ubicación técnica', 'Número de serie',
       'Pto.tbjo.responsable', 'Dirección', 'Latitud', 'Longitud', 'Estatus'],
      dtype='object')


In [449]:
result_df = pd.merge(left = programa, right = nombres[['TecID','Short name','Estatus','City']],left_on="Responsable",right_on="TecID",how="left").drop(columns=['TecID'])

In [450]:
result_df = pd.merge(result_df,politicas[['Equipo','gestora archivo anahi ','Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS']],on="Equipo",how="left")

In [451]:
result_df = pd.merge(result_df,mlocal[['Orden', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado']], on="Orden",how= "left")

In [452]:
result_df = pd.merge(result_df,mforaneo[['Orden','UBICACIÓN', 'ESTATUS COMPRA', 'PROVEEDOR']],on='Orden',how='left')

In [453]:
result_df = pd.merge(left = result_df,right = reprogramaciones[['Orden','Fecha_reprogramada']],on='Orden',how='left')

In [454]:
print(reportes.columns)

Index(['Semana', 'Inicio programado', 'Orden', 'Pto.tbjo.responsable',
       'Cliente', 'Denominación de objeto técnico', 'Número de serie',
       'Equipo', 'Clase de orden', 'N° de Reporte ', 'ESTATUS 19 SEPTIEMBRE',
       'ESTATUS 20 SEPTIEMBRE', 'ESTATUS 22 SEPTIEMBRE',
       'ESTATUS 23 SEPTIEMBRE', 'ESTATUS 26 SEPTIEMBRE',
       'ESTATUS 27 SEPTIEMBRE', 'ESTATUS 28 SEPTIEMBRE',
       'ESTATUS 29 SEPTIEMBRE', 'ESTATUS 03 OCTUBRE', 'ESTATUS 04 OCTUBRE',
       'ESTATUS 05 OCTUBRE', 'ESTATUS 06 OCTUBRE', 'ESTATUS 07 OCTUBRE',
       'ESTATUS 10 OCTUBRE', 'ESTATUS 11 OCTUBRE', 'ESTATUS 12 OCTUBRE',
       'ESTATUS 13 OCTUBRE', 'ESTATUS 14 OCTUBRE', 'ESTATUS 17 OCTUBRE',
       'ESTATUS 18 OCTUBRE', 'ESTATUS 20 OCTUBRE', 'ESTATUS 21 OCTUBRE',
       'ESTATUS 24 OCTUBRE', 'ESTATUS 25 OCTUBRE', 'ESTATUS 26 OCTUBRE',
       'ESTATUS 27 OCTUBRE', 'DETALLE POLITICA', 'COMENTARIO', 'TIPO CLIENTE',
       'MES', 'MES2', 'ANTIGUEDAD', 'GESTORA', 'GPO CLIENTE',
       'FORMATO ESPECIAL', 

In [455]:
result_df = pd.merge(left = result_df, right= reportes[['Orden','ESTATUS 27 OCTUBRE', 'DETALLE POLITICA', 'COMENTARIO']],on='Orden',how='outer')

C:\Users\mruiz\AppData\Local\Temp\ipykernel_15364\3281238980.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  result_df = pd.merge(left = result_df, right= reportes[['Orden','ESTATUS 27 OCTUBRE', 'DETALLE POLITICA', 'COMENTARIO']],on='Orden',how='outer')


In [456]:
result_df = pd.merge(left=result_df,right=ubicaciones[['Equipo','Dirección']],how='left',on='Equipo')

In [457]:
print(result_df.loc[0])

Inicio_programado                                                2024-07-26 00:00:00
Orden                                                                      4096274.0
Plan_Mtto                                                                     8396.0
Responsable                                                                 TECSER24
Status                                                                     BLOQUEADA
Descripción                                               SEGUIDORES DE LA CRUZ 10KW
Cliente                                                                     303371.0
Cliente_Equipo                                            SEGUIDORES DE LA CRUZ 10KW
Número_de_serie                                                                  NaN
Equipo                                                                       12923.0
Clase_de_orden                                                                  ZPM2
Fecha_Liberación                                                 

In [458]:
result_df.assign(Pendiente_Recoger=0)

,Inicio_programado,Orden,Plan_Mtto,Responsable,Status,Descripción,Cliente,Cliente_Equipo,Número_de_serie,Equipo,...,Material Separado,UBICACIÓN,ESTATUS COMPRA,PROVEEDOR,Fecha_reprogramada,ESTATUS 27 OCTUBRE,DETALLE POLITICA,COMENTARIO,Dirección,Pendiente_Recoger
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,NaN,2024-07-26,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTO,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,NaN,2023-12-12,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTO,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,NaN,2023-10-12,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTO,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,NaN,2023-09-13,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTO,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,NaN,2023-09-11,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40625,NaT,7958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,0
40626,NaT,7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,REPROGRAMACION DE SERVICIO,NaN,0
40627,NaT,4100049-4100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Pendiente por facturar,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,0
40628,NaT,6462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,Pendiente por bloque completo,Corporativo,SIN COMENTARIO,NaN,0


In [459]:
result_df['Pendiente_Recoger'] = np.where(result_df['Inicio programado2'].isnull(),result_df['ESTATUS COMPRA'],result_df['Inicio programado2'])

In [460]:
result_df.assign(reprogramacion=0)

,Inicio_programado,Orden,Plan_Mtto,Responsable,Status,Descripción,Cliente,Cliente_Equipo,Número_de_serie,Equipo,...,UBICACIÓN,ESTATUS COMPRA,PROVEEDOR,Fecha_reprogramada,ESTATUS 27 OCTUBRE,DETALLE POLITICA,COMENTARIO,Dirección,Pendiente_Recoger,reprogramacion
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,2024-07-26,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",NaN,0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTO,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,2023-12-12,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",NaN,0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTO,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,2023-10-12,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",NaN,0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTO,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,2023-09-13,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",NaN,0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTO,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,2023-09-11,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40625,NaT,7958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,NaN,0
40626,NaT,7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,REPROGRAMACION DE SERVICIO,NaN,NaN,0
40627,NaT,4100049-4100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,Pendiente por facturar,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,NaN,0
40628,NaT,6462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaT,Pendiente por bloque completo,Corporativo,SIN COMENTARIO,NaN,NaN,0


In [461]:
reprogramaciones = result_df.copy()

In [462]:
result_df['reprogramacion'] = np.where(result_df['Inicio_programado']==reprogramaciones['Fecha_reprogramada'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_reprogramada'].isnull(),"O.T. Agregada",result_df['reprogramacion'])

In [463]:
print(result_df['reprogramacion'].unique())

['Sin_Reprogramar' 'O.T. Agregada' 'Reprogramada']


In [464]:
result_df.assign(Politica=0)

,Inicio_programado,Orden,Plan_Mtto,Responsable,Status,Descripción,Cliente,Cliente_Equipo,Número_de_serie,Equipo,...,ESTATUS COMPRA,PROVEEDOR,Fecha_reprogramada,ESTATUS 27 OCTUBRE,DETALLE POLITICA,COMENTARIO,Dirección,Pendiente_Recoger,reprogramacion,Politica
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,2024-07-26,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",NaN,Sin_Reprogramar,0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTO,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,2023-12-12,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",NaN,Sin_Reprogramar,0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTO,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,2023-10-12,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",NaN,Sin_Reprogramar,0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTO,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,2023-09-13,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",NaN,Sin_Reprogramar,0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTO,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,2023-09-11,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",NaN,Sin_Reprogramar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40625,NaT,7958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,NaN,O.T. Agregada,0
40626,NaT,7939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,Archivo,SIN DETALLE DE POLITICA,REPROGRAMACION DE SERVICIO,NaN,NaN,O.T. Agregada,0
40627,NaT,4100049-4100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,Pendiente por facturar,SIN DETALLE DE POLITICA,SIN COMENTARIO,NaN,NaN,O.T. Agregada,0
40628,NaT,6462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,Pendiente por bloque completo,Corporativo,SIN COMENTARIO,NaN,NaN,O.T. Agregada,0


In [465]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")

In [466]:
print(result_df.shape[0])
print(programa.shape[0])

40630
40552


In [467]:
result_df.columns

Index(['Inicio_programado', 'Orden', 'Plan_Mtto', 'Responsable', 'Status',
       'Descripción', 'Cliente', 'Cliente_Equipo', 'Número_de_serie', 'Equipo',
       'Clase_de_orden', 'Fecha_Liberación', 'Fecha_creación', 'Material',
       'Short name', 'Estatus', 'City', 'gestora archivo anahi ', 'Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS',
       'Inicio programado2', 'Fecha de salida', 'Comentario',
       'Material Separado', 'UBICACIÓN', 'ESTATUS COMPRA', 'PROVEEDOR',
       'Fecha_reprogramada', 'ESTATUS 27 OCTUBRE', 'DETALLE POLITICA',
       'COMENTARIO', 'Dirección', 'Pendiente_Recoger', 'reprogramacion',
    

In [499]:
reprogramaciones = result_df[result_df['reprogramacion'].isin(['Reprogramada'])].drop(columns={'Material Separado', 'UBICACIÓN',
       'ESTATUS COMPRA', 'PROVEEDOR','Cliente_Equipo', 'Número_de_serie', 'Equipo',
       'Clase_de_orden', 'Fecha_Liberación', 'Fecha_creación', 'Material',
       'Short name', 'Estatus', 'City', 'gestora archivo anahi ', 'Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS',
       'Inicio programado2', 'Fecha de salida', 'Comentario', 'ESTATUS 27 OCTUBRE', 'DETALLE POLITICA',
       'COMENTARIO', 'Dirección', 'Pendiente_Recoger',
       'Politica'})

if datetime.today().strftime('%Y-%m-%d') == reprogramaciones_['Dia_Reprogramado'].iloc[-1]:
       pass
else:
       reprogramaciones = reprogramaciones.assign(Dia_Reprogramado=datetime.today().strftime('%Y-%m-%d'))
       reprogramaciones = reprogramaciones.append(reprogramaciones_)
       print(reprogramaciones.shape[0])
       print(reprogramaciones.columns)
       print(reprogramaciones)

In [501]:
print(reprogramaciones.shape[0])
print(reprogramaciones.columns)
print(reprogramaciones)

19
Index(['Inicio_programado', 'Orden', 'Plan_Mtto', 'Responsable', 'Status',
       'Descripción', 'Cliente', 'Fecha_reprogramada', 'reprogramacion'],
      dtype='object')
     Inicio_programado      Orden  Plan_Mtto Responsable    Status  \
2355        2023-01-10  4093291.0        NaN     TECSER9  LIBERADA   
3842        2022-11-24  4096747.0     4268.0    TECSER17  LIBERADA   
4033        2022-11-18  4095635.0     7811.0     TECSER9  LIBERADA   
4062        2022-11-18  4097558.0     8435.0     TECSER9  LIBERADA   
4343        2022-11-12  4093493.0     7855.0    TECSER30  LIBERADA   
4352        2022-11-11  4101024.0        NaN    TECSER30  LIBERADA   
4502        2022-11-09  4086044.0     7479.0    TECSER13  LIBERADA   
4593        2022-11-08  4092352.0     4943.0    TECSER30  LIBERADA   
4594        2022-11-08  4100519.0     1355.0    TECSER30  LIBERADA   
4745        2022-11-05  4100938.0        NaN    TECSER24  LIBERADA   
4757        2022-11-05  4101044.0        NaN    TECSER14

In [495]:
print(datetime.today())

2022-11-03 17:21:54.576917


In [494]:
os.remove(current_path + "\\" + "Reprogramaciones.xlsx")
path = current_path + "\\" + "Reprogramaciones.xlsx"   
reprogramaciones.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    reprogramaciones.to_excel(writer,index=False,sheet_name="DB")

In [428]:
os.remove(current_path + "\\" + "Main.xlsx")
path = current_path + "\\" + "Main.xlsx"   
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")